In [ ]:
#---#| default_exp match.match_utils

# PSM Match functionalities

In [ ]:
from alpharaw.match.match_utils import *
import numpy as np

In [ ]:
#| hide
spec_masses = np.array([100, 100.1, 199.9, 200.1, 300])
query_masses = np.array([10,100.0, 200, 300, 400])
Da_tols = np.ones_like(query_masses)*0.2
first_indices, last_indices = match_profile_peaks(
    spec_masses, 0, query_masses, Da_tols,
)
print(first_indices, last_indices)
assert np.all(first_indices==[-1, 0,  2,  4, -1])
assert np.all(last_indices==[-1, 1,  3,  4, -1])

[-1  0  2  4 -1] [-1  1  3  4 -1]


In [ ]:
#| hide
#unittests
spec_masses = np.arange(10, dtype=float)*100
query_masses = spec_masses
Da_tols = query_masses*20*1e-6
idxes = match_closest_peaks(spec_masses, 0, query_masses, Da_tols)
assert np.all(
    np.arange(10, dtype=np.int32)==
    idxes
)

In [ ]:
#| hide
#unittests
spec_masses = np.arange(10, dtype=float)*100
query_masses = spec_masses[:5]
Da_tols = query_masses*20*1e-6
spec_intens = np.ones_like(spec_masses)
idxes = match_highest_peaks(spec_masses, spec_intens, query_masses, Da_tols)
assert np.all(
    np.arange(5, dtype=np.int32)==
    idxes
)

In [ ]:
#| hide
#unittests
spec_masses = np.arange(10, dtype=float)*100
query_masses = spec_masses.copy()[:5]
query_masses[1]+=0.01 # -1
query_masses[3]+=0.003 # spec_masses[3]
query_masses[5]-=0.0099 # spec_masses[5]
query_masses[6]+=0.02 #-1
spec_masses[7] = spec_masses[8]-0.007 # matched[7] = -1
spec_masses[8] += 0.01 # matched[8] = 7 as 7 is closer
Da_tols = query_masses*20*1e-6
assert np.allclose(
    match_closest_peaks(spec_masses, 0, query_masses, Da_tols),
    [0, -1,  2,  3,  4]
)

In [ ]:
#| hide
#unittests
spec_masses = np.arange(10, dtype=float)*100
query_masses = np.arange(20).reshape((10,2))[:5]*100
Da_tols = query_masses*20*1e-6
target = np.arange(20, dtype=np.int32).reshape((10,2))[:5]
target[10:] = -1
assert np.allclose(
    target,
    match_closest_peaks(spec_masses, 0, query_masses, Da_tols)
)